In [17]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import json

In [5]:
dataset = load_dataset("banking77")

BASE_DATASET = 'card'
DRIFT_DATASET = 'new card'
EVOLUTION_DATASET = 'new'

Using custom data configuration default
Reusing dataset banking77 (/Users/pgupta282/.cache/huggingface/datasets/banking77/default/1.1.0/ff44c4421d7e70aa810b0fa79d36908a38b87aff8125d002cd44f7fcd31f493c)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
fields = pd.read_csv('data/fields.csv')
fields.head()

,label,intent,dataset
0,0,activate_my_card,card
1,1,age_limit,general
2,2,apple_pay_or_google_pay,new
3,3,atm_support,card
4,4,automatic_top_up,topup


In [26]:
def create_rasa_data(dataset_name):
    labels = fields[fields['dataset'] == dataset_name]['label'].tolist()
    intents = []

    # Creating nlu.md
    with open(f'output/nlu_{dataset_name}.md', 'w') as f:
        for label in tqdm(labels):
            intent = fields[fields['label'] == label]['intent'].values[0]
            f.write(f'## intent: {intent}')
            for obj in dataset['train']:
                if obj['label'] == label:
                    f.write(f'- {obj["text"]}\n')
            intents.append(intent)

    # Creating stories.md
    with open(f'output/stories_{dataset_name}.md', 'w') as f:
        for intent in intents:
            f.write(f'## {intent}\n* {intent}\n  - utter_{intent}\n\n')

    # Creating domain.yml
    with open(f'output/domain_{dataset_name}.yml', 'w') as f:
        f.write('intents:\n')
        for intent in intents:
            f.write(f'  - {intent}\n')
        f.write('\nresponses:\n')
        for intent in intents:
            f.write(f'  - utter_{intent}:\n  - text: "Text for {intent}"\n')

create_rasa_data('card')
create_rasa_data('new card')
create_rasa_data('new')

100%|██████████| 3/3 [00:01<00:00,  2.20it/s]
